In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load Data Sources into Pandas Dataframes

In [2]:
# Load your data sets
df_icd = pd.read_pickle('mimic_iv_extract/df_icd_codes_with_description.pkl')
df_history = pd.read_pickle('mimic_iv_extract/df_past_medical_history.pkl')
df_lab_events = pd.read_pickle("mimic_iv_extract/df_lab_events.pkl")
df_ckd_lab_items = pd.read_pickle("mimic_iv_extract/df_ckd_lab_items.pkl")

# Define Helper Functions

In [3]:
# helper function for dropping sparsely populated columns from dataframes with specified threshold

def drop_sparse_columns(df, sparsity_threshold):
    # calculate a series of percentage NaN values in each column
    pct_NaN = df.isna().mean()
    
    # identify 'sparse columns' NaN percentage >= sparsity_threshold
    sprs_Cols = pct_NaN[pct_NaN >= sparsity_threshold].index
    
    # drop sparse columns from the frame and return
    df.drop(columns=sprs_Cols, inplace=True)
    return df

In [4]:
# helper function for performing ANOVA analysis on a dataframe for a specified target column and feature set

def ANOVA(df, targ_col, features):
    # Dictionary to hold ANOVA results
    anova_results = {}
    
    for feature in features:
        # Only proceed if the column exists in the DataFrame to avoid KeyError
        if feature in df.columns:
            try:
                # Prepare the formula for the OLS model. Using 'Q()' around feature names to handle numeric column names.
                formula = f'Q("{feature}") ~ C({targ_col})'
                
                # Fit the model
                model = ols(formula, data=df).fit()
                
                # Perform ANOVA
                anova_table = sm.stats.anova_lm(model, typ=2)
                
                # Extract the p-value for the group effect
                p_value = float(anova_table["PR(>F)"][0])
                
                # Store the result
                anova_results[feature] = p_value
            except ValueError:
                continue  # Skip ValueErrors in sparse features to the next iteration
        else:
            anova_results[feature] = 'Feature column not found'

    # sort anova results in order of increasing p-value
    anova_results = sorted(anova_results.items(), key=lambda item: item[1])

    return anova_results

# Format Target Dataframe Containing Hospital Admissions Related to CKD Diagnoses

In [5]:
# subset the CKD patients using ICD-9 codes (585)
df_target = df_icd[df_icd['icd_code'].str.startswith('585')]

# ensure the target column is of type 'category' for ANOVA analysis
df_target['icd_code'] = df_target['icd_code'].astype('category')

df_target.head()

C:\Users\zdana\AppData\Local\Temp\ipykernel_20408\1649109332.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target['icd_code'] = df_target['icd_code'].astype('category')


,hadm_id,subject_id,icd_code,seq_num,long_title
222,24522342,10031358,5859,36,"Chronic kidney disease, unspecified"
262,20755971,10038081,5859,29,"Chronic kidney disease, unspecified"
401,22394571,10098215,5859,32,"Chronic kidney disease, unspecified"
770,21801929,10173670,5859,28,"Chronic kidney disease, unspecified"
1672,29393377,10332722,5853,28,"Chronic kidney disease, Stage III (moderate)"


# Format Features Frames

In [6]:
# cast itemid column to string in CKD lab items frame
df_ckd_lab_items['itemid'] = df_ckd_lab_items['itemid'].astype(str)

In [7]:
# create a frame of all lab events to use as features for ANOVA
df_lab_features = df_lab_events.copy()

# convert lab features value column to float
df_lab_features['value'] = pd.to_numeric(df_lab_features['value'], errors='coerce')

# pivot lab features DataFrame such that each unique lab item ID becomes its own column with the mean value for the corresponding subject and hadm ID
df_lab_features = df_lab_features.pivot_table(index=['subject_id', 'hadm_id'], columns='itemid', values='value', aggfunc='mean').reset_index()

# merge the DataFrames on 'hadm_id' and 'subject_id'
df_lab_features_mrg = pd.merge(df_target, df_lab_features, on=['hadm_id', 'subject_id'])

# drop columns with all NaN values
df_lab_features_mrg = drop_sparse_columns(df_lab_features_mrg, 1)

# map column names to string
df_lab_features_mrg.columns = df_lab_features_mrg.columns.map(str)

df_lab_features_mrg.head(3)

,hadm_id,subject_id,icd_code,seq_num,long_title,50808,50811,50853,50861,50862,...,51272,51273,51282,51283,51284,51300,51301,51492,51493,51494
0,24522342,10031358,5859,36,"Chronic kidney disease, unspecified",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.114286,NaN,NaN,NaN
1,20755971,10038081,5859,29,"Chronic kidney disease, unspecified",1.137500,NaN,NaN,9.8,3.525,...,NaN,NaN,0.12,4.9,NaN,NaN,5.534783,30.0,21.333333,NaN
2,22394571,10098215,5859,32,"Chronic kidney disease, unspecified",1.186957,11.533333,NaN,25.6,2.700,...,NaN,NaN,NaN,NaN,NaN,NaN,9.096629,30.0,16.666667,NaN


# Perform ANOVA Analysis

In [8]:
# isolate a list of column names to evalute as features from the df_lab_features_mrg frame
features = list(df_lab_features_mrg.columns[df_lab_features_mrg.columns.get_loc('long_title') + 1:])

anova_lab_features = ANOVA(df_lab_features_mrg, 'icd_code', features)

C:\Users\zdana\AppData\Local\Temp\ipykernel_20408\317416107.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = float(anova_table["PR(>F)"][0])
C:\Users\zdana\AppData\Local\Temp\ipykernel_20408\317416107.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = float(anova_table["PR(>F)"][0])
C:\Users\zdana\AppData\Local\Temp\ipykernel_20408\317416107.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = flo

In [9]:
# Display the results
for tup in anova_lab_features:
    print(f"{df_ckd_lab_items.loc[df_ckd_lab_items['itemid'] == tup[0], 'label'].values[0]}:\n    p-value = {tup[1]}")

Chloride:
    p-value = 0.0
Creatinine:
    p-value = 0.0
Phosphate:
    p-value = 0.0
Urea_Nitrogen:
    p-value = 0.0
MCV:
    p-value = 2.1473631214254582e-282
MCHC:
    p-value = 6.20979821678653e-281
Hemoglobin:
    p-value = 8.230572969674965e-255
Potassium:
    p-value = 1.5448844327362312e-246
Sodium:
    p-value = 7.174852529422103e-243
Bicarbonate:
    p-value = 1.6236057454893205e-190
Protein:
    p-value = 1.4646527499097125e-146
MCH:
    p-value = 5.58863911876911e-48
25-OH_Vitamin_D:
    p-value = 1.875751483511302e-42
Reticulocyte_Count,_Manual:
    p-value = 8.350544106151282e-40
Urea_Nitrogen,_Urine:
    p-value = 4.727064161484161e-32
Platelet_Count:
    p-value = 4.186215823027653e-28
Total_Protein,_Urine:
    p-value = 3.9539730475227007e-26
Uric_Acid:
    p-value = 2.4953930774451954e-23
White_Blood_Cells:
    p-value = 7.544174589898482e-19
Protein/Creatinine_Ratio:
    p-value = 5.587634233032481e-17
Free_Calcium:
    p-value = 7.85012705266186e-15
Creatinine,_Ur